In [2]:
!pip install -U "tensorflow-text==2.15.*"

In [3]:
import tensorflow as tf

2024-04-05 15:15:18.487829: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-05 15:15:18.487975: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-05 15:15:18.637973: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [4]:
import tensorflow_hub as hub

In [5]:
preprocess_url = "https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3"
encoder_url = "https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/3"

In [6]:
preprocess_url = "https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3"
encoder_url = "https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/3"

In [7]:
!pip install -q -U "tensorflow-text==2.15.*"

In [8]:
import tensorflow_text as text

In [9]:
bert_preprocess = hub.KerasLayer(preprocess_url)

In [10]:
import pandas as pd
df = pd.read_csv('/kaggle/input/cleaned-file/cleaned_file1.csv')
df.head(1)

,text,label,category
0,FLASH SALE | LIMITED TIME ONLY Shop Now,1,Urgency


In [11]:
import numpy as np
df.label.nunique(dropna = True)

2

In [12]:
x = df['text']
y = df.label
y= np.asarray(y, dtype=np.float32)

In [13]:
from sklearn.model_selection import train_test_split
x_train , x_test, y_train, y_test = train_test_split(x,y,test_size=0.2)

In [14]:
bert_model = hub.KerasLayer(encoder_url)

In [15]:
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
preprocessed_text = bert_preprocess(text_input)
bert_results = bert_model(preprocessed_text)
pooled_output = bert_results['pooled_output']
# neural network layers
# dense_layer_1 = tf.keras.layers.Dense(600, activation='relu', name='dense_layer_1')(pooled_output)
# dense_layer_2 = tf.keras.layers.Dense(400, activation='relu', name='dense_layer_2')(dense_layer_1)
dense_layer_3 = tf.keras.layers.Dense(400, activation='relu', name='dense_layer_3')(pooled_output)
dense_layer_4 = tf.keras.layers.Dense(200, activation='relu', name='dense_layer_4')(dense_layer_3)
dropout_layer_1 = tf.keras.layers.Dropout(0.2, name='dropout_1')(dense_layer_4)
output_layer = tf.keras.layers.Dense(1, activation='sigmoid', name='output')(dropout_layer_1)

bert_model.trainable = True

model = tf.keras.Model(inputs=[text_input], outputs = [output_layer])

In [16]:
from tensorflow.keras.optimizers import Adam

opt = Adam(learning_rate=1e-4)
model.compile(optimizer=opt,
              loss = 'binary_crossentropy',
              metrics = ['accuracy'])

In [17]:
model.fit(x_train,y_train,epochs=3)

Epoch 1/3


I0000 00:00:1712330208.715427     110 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


59/59 [==============================] - 96s 805ms/step - loss: 0.1871 - accuracy: 0.9305
Epoch 2/3
59/59 [==============================] - 49s 825ms/step - loss: 0.0879 - accuracy: 0.9713
Epoch 3/3
59/59 [==============================] - 49s 834ms/step - loss: 0.0488 - accuracy: 0.9889


In [18]:
model.evaluate(x_test,y_test)

15/15 [==============================] - 6s 306ms/step - loss: 0.1066 - accuracy: 0.9725


[0.10661831498146057, 0.9724576473236084]

In [19]:
import joblib

In [20]:
joblib.dump(model, '/kaggle/working/nlp_model.joblib')

['/kaggle/working/nlp_model.joblib']

In [30]:
text = ['Get Upto Extra 10% Off on 4999 and above. Max Discount Rs.1000',
       'Easy 10 days return and exchange. Return Policies may vary based on products and promotions. For full details on our Returns Policies, please click here․',
       'Pair them up with stylish hoodies for women and hoodies for men.'
       ]

In [32]:
predicted = model.predict(text)
for i in predicted:
    if(i>=0.5):
        print("A dark pattern")
    else : 
        print("Not a dark pattern")

1/1 [==============================] - 0s 53ms/step
A dark pattern
A dark pattern
Not a dark pattern
